#Import

In [52]:
import torch
from torch import nn, optim 
import torch.nn.functional as F 
from torchvision import datasets, transforms 
import matplotlib.pyplot as plt
from tqdm import tqdm

if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc. 
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [53]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

In [54]:
trainset = datasets.MNIST('~/MNIST_data/', download=True, train=True, transform=transform)
testset = datasets.MNIST('~/MNIST_data/', download=True, train=False, transform=transform)

#Data Processing

In [55]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)

#Building Network

In [56]:
class Model(nn.Module):

  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(1, 32, 3) # input is 1 image, 32 output channels, 5x5 kernel / window
    self.conv2 = nn.Conv2d(32, 64, 3) # input is 32, bc the first layer output 32. Then we say the output will be 64 channels, 5x5 kernel / window
    self.conv3 = nn.Conv2d(64, 128, 3)

    x = torch.randn(28,28).view(-1,1,28,28)
    self._to_linear = None
    self.convs(x)

    self.fc1 = nn.Linear(self._to_linear, 512) #flattening.
    self.fc2 = nn.Linear(512, 10) # 512 in, 2 out bc we're doing 2 classes (dog vs cat).

  def convs(self, x):
    # max pooling over 2x2
    x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
    x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
    x = F.max_pool2d(F.relu(self.conv3(x)), (2, 2))

    if self._to_linear is None:
        self._to_linear = x[0].shape[0]*x[0].shape[1]*x[0].shape[2]
    return x

  def forward(self, x):
    x = self.convs(x)
    x = x.view(-1, self._to_linear)  # .view is reshape ... this flattens X before 
    x = F.relu(self.fc1(x))
    x = self.fc2(x) # bc this is our output layer. No activation here.
    return F.softmax(x, dim=1)

model = Model().to(device)

In [57]:
import torch.optim as optim
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
epochs = 10
print(model)

Model(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=128, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=10, bias=True)
)


#Training the Model

In [58]:
for epoch in range(epochs):
  for data in trainloader:
    x, y = data
    x.view(-1, 28*28)
    x.to(device)
    y.to(device)
    optimizer.zero_grad(0)
    y_hat = model(x.view(-1,1,28,28).to(device))
    loss = loss_function(y_hat.to(device), y.to(device))
    loss.backward()
    optimizer.step()

#Results / testing ze model

In [59]:
correct = 0
total = 0
with torch.no_grad():
  for data in testloader:
    x, y = data
    output = model(x.view(-1, 1, 28, 28).to(device))
    for idx, i in enumerate(output):
        if torch.argmax(i) == y[idx]:
            correct +=1
        total +=1
print(f'accuracy: {round(correct/total, 3)}')

accuracy: 0.978


In [61]:
print(total)

10000


In [60]:
print(correct)

9780
